In [1]:
import cv2
from matplotlib import pyplot as plt
import os
import numpy as np
current_dir = os.getcwd()
current_dir

'/home/kacper/semestr_6/ZaawansowaneAlgorytmyWizyjne/lab2'

In [2]:
def median_and_erote_dilate(I: np.array) -> np.array:
    I_copy = I.copy()
    I_copy = cv2.medianBlur(I_copy, 5)
    kernel = np.ones((5, 5), np.uint8)
    I_copy = cv2.dilate(I_copy, kernel)
    I_copy = cv2.erode(I_copy, kernel)
    return I_copy

def calculate_true_positive(I: np.array, I_copy: np.array) -> int:
    TP = 0
    TP_M = lambda x,y: np.logical_and((x == 255), (y == 255))
    TP_S = np.sum(TP_M(I, I_copy)) # sum of the elements in the matrix
    TP += TP_S
    return TP

def calculate_false_negative(I: np.array, I_copy: np.array) -> int:
    FN = 0
    FN_M = lambda x,y: np.logical_and((x == 255), (y == 0))
    FN_S = np.sum(FN_M(I, I_copy)) # sum of the elements in the matrix
    FN += FN_S
    return FN

def calculate_true_negative(I: np.array, I_copy: np.array) -> int:
    TN = 0
    TN_M = lambda x,y: np.logical_and((x == 0), (y == 0))
    TN_S = np.sum(TN_M(I, I_copy)) # sum of the elements in the matrix
    TN += TN_S
    return TN

def calculate_false_positive(I: np.array, I_copy: np.array) -> int:
    FP = 0
    FP_M = lambda x,y: np.logical_and((x == 0), (y == 255))
    FP_S = np.sum(FP_M(I, I_copy)) # sum of the elements in the matrix
    FP += FP_S
    return FP

In [15]:
def pipeline(folder_name: str, threshhold_real: int = 15, threshold_groundtruth:int = 15) -> None:

    step =  1
    previous = cv2.imread (f"{current_dir}/{folder_name}/input/in000300.jpg")
    previous = cv2.cvtColor(previous, cv2.COLOR_BGR2GRAY).astype('int')
    true_positive_array = []
    false_negative_array = []
    true_negative_array = []
    false_positive_array = []   


    for i in range (300 ,1100,step) :
        I = cv2.imread (f"{current_dir}/{folder_name}/input/in%06d.jpg" % i )
        I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype('int')
        I_diff = np.abs(I_gray - previous).astype('uint8')
        T, I_binarized = cv2.threshold(I_diff, threshhold_real, 255, cv2.THRESH_BINARY) 
        I_diff = I_binarized.astype('uint8')
        I_diff = median_and_erote_dilate(I_diff).astype('uint8')
        retval , labels , stats , centroids = cv2 . connectedComponentsWithStats (I_diff)
        cv2.imshow("I_diff", I_diff)

        I_VIS = I.copy()
        if stats.shape[0] > 1:
            tab = stats[1:, 4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]), (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]), (255,0, 0), 2)
            cv2.putText(I_VIS, f"#{stats[pi, 4]}", (stats[pi,0], stats[pi, 1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
            cv2.putText(I_VIS, f"#{pi}", (int(centroids[pi, 0]), int(centroids[pi, 1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
        # cv2.imshow("I", I_VIS)
        # cv2.waitKey(5)

        groundtruth = cv2.imread (f"{current_dir}/{folder_name}/groundtruth/gt%06d.png" % i, cv2.IMREAD_GRAYSCALE )
        groundtruth_binarized = (groundtruth > threshold_groundtruth).astype('uint8') * 255
        # cv2.imshow("groundtruth", groundtruth_binarized)
        # cv2.imshow("I", I_VIS)
        cv2.waitKey(5)
        TP = calculate_true_positive(groundtruth_binarized, I_diff)
        FN = calculate_false_negative(groundtruth_binarized, I_diff)
        TN = calculate_true_negative(groundtruth_binarized, I_diff)
        FP = calculate_false_positive(groundtruth_binarized, I_diff)
        true_positive_array.append(TP)
        false_negative_array.append(FN)
        true_negative_array.append(TN)
        false_positive_array.append(FP)
        previous = I_gray


    cv2.destroyAllWindows()
    precision = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_negative_array))
    print("Preicison: ", precision)
    recall = np.sum(true_positive_array) / (np.sum(true_positive_array) + np.sum(false_positive_array))
    print("Recall: ", recall)
    f1 = 2 * ((precision * recall) / (precision + recall))
    print("F1: ", f1)

In [16]:
pipeline("pedestrian", 10, 5)

Preicison:  0.6665692313891691
Recall:  0.7838583701769245
F1:  0.7204714951130776


In [30]:
pipeline("highway", 13, 5)

Preicison:  0.17535860877698592
Recall:  0.9674270754934787
F1:  0.296900229652464


In [36]:
pipeline("office", 10, 5)

Preicison:  0.14079558572189144
Recall:  0.4562059142321633
F1:  0.2151812982347672
